In [6]:
from river import metrics, evaluate
from river.datasets import Phishing
from river_torch.classification import Classifier
import torch
from tqdm import tqdm

import matplotlib.pyplot as plt

In [7]:
n_samples = 10000
dataset = Phishing()

metric = metrics.Accuracy()

def build_fn(n_features, n_classes=1):
    net = torch.nn.Sequential(
        torch.nn.Linear(n_features, n_classes),
        torch.nn.Sigmoid()
    )
    return net

optimizer_fn = torch.optim.SGD
model = Classifier(build_fn=build_fn,loss_fn="binary_cross_entropy",optimizer_fn=optimizer_fn)

In [8]:
for x,y in dataset:
    y_pred = model.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model.learn_one(x, y)    # make the model learn
print(f'Accuracy: {metric.get()}')

Accuracy: 0.4192
